In [2]:
import pandas as pd
import numpy as np

C:\Users\enzok\AppData\Local\Temp\ipykernel_48084\2937650959.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
df_train = pd.read_csv('for_students/waiting_times_train.csv')

In [10]:
date_time = df_train["DATETIME"]

from datetime import datetime

# Fonction pour convertir la date en une liste avec le jour de la semaine, le mois et l'année
def convert_date(date_str):
    date_obj = datetime.strptime(date_str, '%Y-%m-%d')
    jour_semaine = date_obj.strftime('%A')  # Jour de la semaine (nom complet)
    mois = date_obj.strftime('%B')  # Mois (nom complet)
    annee = date_obj.year  # Année
    return [jour_semaine, mois, annee]

times = []
years = []
months = []
days_week = []

for date in date_time:
    day, month, year = convert_date(date[:10])
    years.append(year)
    months.append(month)
    days_week.append(day)
    time = date[11:16]
    times.append(int(time[:2]) * 4 + int(time[3:5]) // 15)

times = np.array(times) - np.min(times)

encoded_years = pd.get_dummies(years)
encoded_months = pd.get_dummies(months)
encoded_days_week = pd.get_dummies(days_week)
encoded_times = pd.get_dummies(times)

print(encoded_days_week)
print(encoded_times)
print(encoded_years)
print(encoded_months)

       Friday  Monday  Saturday  Sunday  Thursday  Tuesday  Wednesday
0       False   False      True   False     False    False      False
1       False   False     False    True     False    False      False
2       False   False      True   False     False    False      False
3       False   False     False    True     False    False      False
4       False   False     False   False     False    False       True
...       ...     ...       ...     ...       ...      ...        ...
37013   False   False     False   False      True    False      False
37014   False   False     False   False     False    False       True
37015   False   False     False   False      True    False      False
37016   False   False     False    True     False    False      False
37017    True   False     False   False     False    False      False

[37018 rows x 7 columns]
          0      1      2      3      4      5      6      7      8      9   \
0      False  False  False  False  False  False  False 

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from sklearn.svm import SVC
import torch

class SVMLayer(nn.Module):
    def __init__(self, in_features, num_classes):
        super(SVMLayer, self).__init__()
        self.svm = SVC(kernel='rbf')  # Créer une instance de SVM à noyau

    def forward(self, x):
        # Convertir les données d'entrée en numpy array
        x_numpy = x.detach().numpy()
        # Former la SVM avec les données d'entrée
        self.svm.fit(x_numpy)
        # Prédire les classes avec la SVM
        predictions = self.svm.predict(x_numpy)
        # Convertir les prédictions en tensor PyTorch
        predictions_tensor = torch.tensor(predictions)
        return predictions_tensor

# Définir la classe de votre modèle
class MonModele(nn.Module):
    def __init__(self, in_features, num_classes):
        super(MonModele, self).__init__()
        self.fc1 = nn.Linear(in_features, 256)
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, 256)
        self.fc4 = nn.Linear(256, 256)
        self.dropout = nn.Dropout(0.5)
        self.svm_layer = SVMLayer(256, num_classes)  # Ajouter la couche SVM personnalisée

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.dropout(x)
        x_svm = self.svm_layer(x)  # Obtenir les prédictions SVM
        return x_svm

# Instancier votre modèle
in_features =  # définir le nombre d'entités en entrée de votre modèle
num_classes = 10  # définir le nombre de classes pour la classification
model = MonModele(in_features, num_classes)

import torch.optim as optim

# Définir une fonction de perte
criterion = nn.CrossEntropyLoss()

# Définir un optimiseur
optimizer = optim.Adam(model.parameters(), lr=0.001)

from torch.utils.data import DataLoader, TensorDataset

# Créer un TensorDataset à partir de vos données d'entraînement
train_dataset = TensorDataset(X_train, y_train)

# Créer un DataLoader pour les données d'entraînement avec une taille de lot de 64
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Boucle d'entraînement
num_epochs = 10
losses = []
for epoch in range(num_epochs):
    model.train()  # Mettre le modèle en mode entraînement
    running_loss = 0.0
    for inputs, labels in train_loader:  # Supposons que vous utilisez un DataLoader pour charger vos données
        optimizer.zero_grad()  # Remettre à zéro les gradients
        outputs = model(inputs)  # Propagation avant
        loss = criterion(outputs, labels)  # Calcul de la perte
        loss.backward()  # Rétropropagation
        optimizer.step()  # Mise à jour des poids
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)
    losses.append(epoch_loss)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

print('Finished Training')

import matplotlib.pyplot as plt

plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.show()